### Download Financial Statements with Augmented Master Index (Includes SIC)

###### Objective: develop basic script to allow querying by SICs, Company Names, and Dates

###### Notes: the script works with a rudimental UI. Future improvements will need to include:

- Ability to input a date range in the query rather than entering single dates
- Improved error handling functionality to force query inputs within the constraints
- An improved UI
- Ability to download financial statements without manually picking reports for each company

Ended 10/3/19 - will continue development in duplicate script named "query_statements_improved"

In [1]:
import pandas as pd
import edgar
import psycopg2
import os
import glob
import numpy as np

### Establish and Test DB Connection  

You will have to enter DB login credentials here:

In [2]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Paolino1")

In [3]:
cur = conn.cursor()

In [4]:
cur.execute('SELECT * FROM NEW_MASTER LIMIT 20')

In [5]:
query = cur.fetchall()

In [6]:
if len(query) == 20:
    print('DB Connection Succesfully Established')

DB Connection Succesfully Established


### API

In [7]:
# Function to query database by company:

def select_company(company_names):
    
    ## Function to query the index by company name
    ## Expectes list of company names to execute select query
    
    ##Connect to DB:
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Paolino1")
    cur = conn.cursor()
    
    ##Create Query:
    SQL = 'SELECT * FROM NEW_MASTER WHERE Company_Name IN %(list)s;'
    cur.execute(SQL, {
        'list':tuple(company_names),
    })
    
    #Present query results:
    output = cur.fetchall()
    columns = ['CIK', 'Company_Name', 'Filing_Type', 'Filing_Date', 'URL_txt', 'URL_html', 'SIC']
    result = pd.DataFrame(output, columns=columns)
    
    ##Closing DB connection:
    cur.close()
    conn.close()
    
    return result

In [8]:
#Sample query to retrieve index data for Amazon and American Airlines:
df_co_name = select_company(['AMAZON COM INC', 'AMERICAN AIRLINES INC'])

In [9]:
#Checking that both Amazon and AA were found:
df_co_name.Company_Name.unique()

array(['AMERICAN AIRLINES INC', 'AMAZON COM INC'], dtype=object)

In [10]:
# Function to query database by SIC:

def select_SIC(company_sics):
    
    ## Function to query the index by SIC
    
    ##Connect to DB:
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Paolino1")
    cur = conn.cursor()
    
    ##Create Query:
    SQL = 'SELECT * FROM NEW_MASTER WHERE SIC IN %(list)s;'
    cur.execute(SQL, {
        'list':tuple(company_sics),
    })
    
    #Present query results:
    output = cur.fetchall()
    columns = ['CIK', 'Company_Name', 'Filing_Type', 'Filing_Date', 'URL_txt', 'URL_html', 'SIC']
    result = pd.DataFrame(output, columns=columns)
    
    ##Closing DB connection:
    cur.close()
    conn.close()
    
    return result

In [11]:
df_sic = select_SIC(['6172'])

In [12]:
df_sic

,CIK,Company_Name,Filing_Type,Filing_Date,URL_txt,URL_html,SIC
0,101679,USL CAPITAL CORP/,10-K,1995-03-24,edgar/data/101679/0000950149-95-000135.txt,edgar/data/101679/0000950149-95-000135-index.html,6172
1,711513,MCDONNELL DOUGLAS FINANCE CORP /DE,10-K,1995-03-31,edgar/data/711513/0000711513-95-000014.txt,edgar/data/711513/0000711513-95-000014-index.html,6172
2,768170,PITNEY BOWES CREDIT CORP,10-K,1995-03-29,edgar/data/768170/0000768170-95-000003.txt,edgar/data/768170/0000768170-95-000003-index.html,6172
3,789577,TRANS LEASING INTERNATIONAL INC,10-Q,1995-02-14,edgar/data/789577/0000950131-95-000340.txt,edgar/data/789577/0000950131-95-000340-index.html,6172
4,801550,DVI INC,10-Q,1995-02-14,edgar/data/801550/0000897204-95-000010.txt,edgar/data/801550/0000897204-95-000010-index.html,6172
5,857067,INDUSTRIAL FUNDING CORP,10-K,1995-03-01,edgar/data/857067/0000912057-95-001128.txt,edgar/data/857067/0000912057-95-001128-index.html,6172
6,101679,UNITED STATES LEASING INTERNATIONAL INC,10-K,1994-03-29,edgar/data/101679/0000950149-94-000071.txt,edgar/data/101679/0000950149-94-000071-index.html,6172
7,768170,PITNEY BOWES CREDIT CORP,10-K,1994-03-15,edgar/data/768170/0000768170-94-000003.txt,edgar/data/768170/0000768170-94-000003-index.html,6172
8,857067,INDUSTRIAL FUNDING CORP,10-K,1994-02-23,edgar/data/857067/0000912057-94-000649.txt,edgar/data/857067/0000912057-94-000649-index.html,6172
9,1124127,MINT LEASING INC,10-Q,2008-11-19,edgar/data/1124127/0001059016-08-000070.txt,edgar/data/1124127/0001059016-08-000070-index....,6172


In [13]:
# Function to query database by SIC:

# Table DDL:
#CREATE TABLE NEW_MASTER(CIK integer, Company_Name text, 
#Filing_Type text,Filing_Date text,URL_text text, URL_html text, SIC text)

def select_mult_criteria(companies, sics, dates):
    
    ## Function to query the index by companies, sics or years
    
    ##Connect to DB:
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Paolino1")
    cur = conn.cursor()

    print(len(companies))
    print(len(sics))
    print(len(dates))
    
    
    if (len(companies)>0 & len(sics)>0 & len(dates)>0):
        print('Cannot specify both Company Names and SICs')
        pass
        
    elif len(companies)==0 and len(sics)>0 and len(dates)>0:
        SQL = 'SELECT * FROM NEW_MASTER WHERE SIC IN %(sic_list)s AND Filing_Date IN %(dates_list)s;'

        cur.execute(SQL, {'sic_list':tuple(sics),
                         'dates_list':tuple(dates),})
        
    elif len(companies)>0 and len(sics)==0 and len(dates)>0:
        SQL = 'SELECT * FROM NEW_MASTER WHERE Company_Name IN %(co_list)s AND Filing_Date IN %(dates_list)s;'

        cur.execute(SQL, {'co_list':tuple(companies),
                          'dates_list':tuple(dates),})
    
    elif len(companies)==0 and len(sics)>0 and len(dates)==0:
        SQL = 'SELECT * FROM NEW_MASTER WHERE SIC IN %(sic_list)s;'

        cur.execute(SQL, {'sic_list':tuple(sics),})
    
    else:
        print('Query format not complying to standards')
        print('Please make sure to enter some dates')
        pass
    
    #Present query results:
    output = cur.fetchall()
    columns = ['CIK', 'Company_Name', 'Filing_Type', 'Filing_Date', 'URL_txt', 'URL_html', 'SIC']
    result = pd.DataFrame(output, columns=columns)
    
    ##Closing DB connection:
    cur.close()
    conn.close()
    
    return result

### Create Financial Statements
Objective: connect to Edgar and retrieve financial statments for selected companies

Reference for XML Table Schema: https://www.w3schools.com/html/html_tables.asp

In [14]:
import requests
from bs4 import BeautifulSoup
import lxml

In [15]:
def query_results():
    ## Function to retrieve url for 2018 10-K
    ## Expects input from select_company function
    
    #Returning 10-K from 2018. If it does not exist, returning most recent 10-K
    
    print('Welcome to ValuTrack Query Managemnt Tool')
    print('QMT lets you query the Edgar Index by Company Name, SIC Code, and Date')
    print('----------------------------------------------------------------------------------------')
    print('In the next steps you will be asked to formulate a query to retrieve available financial statements')
    print('You can add as many search conditions as you would like')
    print('The query will search for index entries matching all conditions entered')
    print('If you only wish to search by SIC code, leave the steps for Company Name and Year blank')
    print('----------------------------------------------------------------------------------------')
    print('Please make sure to either enter some Companies or SIC codes - Not Both!')
    print('----------------------------------------------------------------------------------------')
    print('Step 1: Company Names')
    print('Please enter company names in list format with names in quotes and separated by commas')
    print('For example: ["MINT LEASING INC", "NewStar Financial, Inc."]')
    
    selected_companies = []
    
    print('Please enter number of companies you want to search for:')
    n = int(input('> '))
    
    for i in range(0,n):
        
        print('Please enter next Company name:')
        ele = str(input('> '))
        
        selected_companies.append(ele)
    
    #Will need to add error handling here
    #print('You have entered')
    
    print('----------------------------------------------------------------------------------------')
    print('Step 2: SIC Codes')
    print('Please enter SIC codes in list format with codes in quotes and separated by commas')
    print('For example: ["6162", "6172"]')
    
    selected_sics = []
    
    print('Please enter number of SIC codes you want to search for:')
    n = int(input('> '))
    
    for i in range(0,n):
        
        print('Please enter next SIC:')
        ele = str(input('> '))
        
        selected_sics.append(ele)
    
    #Will need to add error handling here
    #print('You have entered')
    
    print('----------------------------------------------------------------------------------------')
    print('Step 3: Dates')
    print('Please enter Dates in list format with Dates in quotes and separated by commas')
    print('For example: ["2014-03-31", "2014-03-14"]')
    
    selected_dates = []
    
    print('Please enter number of Dates you want to search for:')
    n = int(input('> '))
    
    for i in range(0,n):
        
        print('Please enter next Date:')
        ele = str(input('> '))
        
        selected_dates.append(ele)
    
    #Will need to add error handling here
    #print('You have entered')
    
    df = select_mult_criteria(selected_companies, selected_sics, selected_dates)
    
    df_10k = df[df['Filing_Type'] == '10-K']
    
    return df_10k

In [16]:
def index_url(df, base_url = r"https://www.sec.gov/Archives/"):
    
    ## Function to retrieve url for 2018 10-K
    ## Expects input from select_company function
    
    #Returning most recent 10-K if it exists
    
    df_recent = df[(df['Filing_Type'] == '10-K')].sort_values('Filing_Date', ascending=False)
        
    if len(df_recent) == 0:
        print('No 10-K available for this company')
        pass
        
    declination = df_recent[['URL_txt']].iloc[0][0]
    
    declination = declination.replace("-","").replace(".txt","/index.json")
    
    return base_url + declination

In [17]:
#def index_url(base_url = r"https://www.sec.gov/Archives/"):
#    urls = ["https://www.sec.gov/Archives/"+url.replace("-","")\
#            .replace(".txt","/index.json") for url in list(df_10k['URL_txt'])]
#        
#    return urls

In [18]:
def download_summary(urls):

    ## Function to retrieve the 10-K Summary which contains links to individual reports
    ## Expects input from index_url function
    
    download = requests.get(co_url).json()
    
    xml_summary = r'https://www.sec.gov'
    
    for item in download['directory']['item']:
        if item['name'] == 'FilingSummary.xml':
            xml_summary += download['directory']['name']+'/' + item['name']

    new_base_url = xml_summary.replace('FilingSummary.xml', '')
    content = requests.get(xml_summary).content
    
    #Using BS4 to parse the XML content
    soup = BeautifulSoup(content, 'lxml')
    reports = soup.find('myreports')
    
    if reports is None:
        print('---------------------')
        print('Sorry, no reports were found for this company!')
        print('Please try a different public company!')
        print('---------------------')
        pass
    
    return new_base_url, reports

In [19]:
def download_summary(co_url):

    ## Function to retrieve the 10-K Summary which contains links to individual reports
    ## Expects input from index_url function
    
    download = requests.get(co_url).json()
    
    xml_summary = r'https://www.sec.gov'
    
    for item in download['directory']['item']:
        if item['name'] == 'FilingSummary.xml':
            xml_summary += download['directory']['name']+'/' + item['name']

    new_base_url = xml_summary.replace('FilingSummary.xml', '')
    content = requests.get(xml_summary).content
    
    #Using BS4 to parse the XML content
    soup = BeautifulSoup(content, 'lxml')
    reports = soup.find('myreports')
    
    if reports is None:
        print('---------------------')
        print('Sorry, no reports were found for this company!')
        print('Please try a different public company!')
        print('---------------------')
        pass
    
    return new_base_url, reports

In [20]:
def show_statements(reports):
    
    #Function to select among available statements contained in the 10-K:
    print('------------------------')
    print('Please select the reports you would like to include in your download')
    print('Copy your selections into the user input provided by the select_reports function')
    print('------------------------')
    
    if reports is None:
        
        print('---------------------')
        print('Sorry, no reports were found for this company!')
        print('Please try a different public company!')
        print('---------------------')
        pass
    
    all_reports = []
    for report in reports.find_all('report'):
        print(report.shortname.text)
        all_reports.append(report.shortname.text)

    return all_reports

In [21]:
def select_reports(all_reports):
    
    all_selections = []
    
    while len(all_selections)<4:
        
        selected = input()
        
        if selected not in all_reports:
            print('Your entry is not valid')
            print('Please copy/paste a report name from the list above.')
            print('Please do not use quotes when entering the report name.')
        else:
            print('-----')
            print('You have entered a valid report!')
            print('Please enter another report from the list above.')
            print('\n')
            all_selections.append(selected)
            
    return all_selections

In [22]:
def statements_urls(new_base_url, reports, short_names):
    
    #Function to create the 4 URL's associated with each Financial Statement Report (see list below)
    #Expectes input from download_summary and pick_statments functions
    
    #Empty dictionary with URL's to downloaded reports
    statements_urls = {}
    
    #Iterating through different reports in the index
    for report in reports.find_all('report'):
        if report.shortname.text in short_names:
            #print('URL for '+report.shortname.text)
            #print(new_base_url+report.htmlfilename.text)
            statements_urls[report.shortname.text] = new_base_url+report.htmlfilename.text

    return statements_urls

In [23]:
def statements_data(statements_urls):
    
    ## Function scrapes financial statements at URL's found and returns a dictionary of unparsed data   
    ## Expects dictionary output by statements_url function
    
    stm_data = {}
    for key, url in statements_urls.items():
        
        statement_data = {}
        statement_data['headers'] = []
        statement_data['sections'] = []
        statement_data['data'] = []

        content = requests.get(url).content
        soup = BeautifulSoup(content, 'html')

        for index, row in enumerate(soup.table.find_all('tr')):

            cols = row.find_all('td')

            if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
                reg_row = [dt.text.strip() for dt in cols]
                statement_data['data'].append(reg_row)

            elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                sec_row = cols[0].text.strip()
                statement_data['sections'].append(sec_row)

            elif (len(row.find_all('th')) != 0):            
                hed_row = [dt.text.strip() for dt in row.find_all('th')]
                statement_data['headers'].append(hed_row)

            else:            
                print('Error.')

        stm_data[key] = statement_data   
        
    return stm_data

In [24]:
def financial_statements(stm_data):
    
    ## Function to create dictionary containing the four reports that make up a financial statement
    ## Expects inputs from statements_data function

    financial_statements = {}
    for key, statement in stm_data.items():

        header =  statement['headers']
        data = statement['data']
        df = pd.DataFrame(data)

        #Some formatting to make sure every df is presented well regardless of dimension
        df = df.replace('[\$,)]','', regex=True )\
                             .replace( '[(]','-', regex=True)\
                             .replace( '', 'NaN', regex=True)\
                             .replace( '\[.*?]', 'NaN', regex=True)

        df.index = df.iloc[:,0]

        df = df.drop(df.columns[0], axis=1)

        df = df.replace('((?:[ a-z ]+\S*\d+|\d\S*[a-z]+)[a-z\d_-]*)', 'NaN', regex=True)\
                        .replace('[^0-9]','', regex=True)\
                        .replace('',np.nan, regex=True)

        df.fillna(value=pd.np.nan, inplace=True)
        df = df.dropna(axis=1, how='all')
        df = df.dropna(axis=0, how='all')

        var = df.shape[1]
        temp = [i for sublist in header for i in sublist]
        df.columns = temp[-var:]
        df.index.name = temp[0]

        df = df.apply(pd.to_numeric)

        #Appending All 4 Statements to Dictionary:
        financial_statements[key]=df
        
    return financial_statements

In [25]:
def run_all():
    
    #Function executes all functions to retrieve Financial Statements for all listed companies
    #Returns a dictionary whose key is the company CIK and values are the 4 statements
    
    df = query_results()
    
    uniques = df.Company_Name.unique()
    df_list = []
    print('----------------------------------------------')
    print('Your query selected the following companies:')
    print(uniques)
    print('----------------------------------------------')
    print('A dataframe containing all index entries matching your query will also be returned')
    print('Along with related financial statements')
    print('----------------------------------------------')
    
    
    
    company_statements_dict = {}
    
    for val in uniques:
        df_list.append(df[df['Company_Name'] == val])
        
    #Executing all functions in sequence:
    for df, val in zip(df_list,uniques):
        print('-------------------------------')
        print('Please select statements for: ')
        print(val)
        print('-------------------------------')
        
        #Retrieving url
        df_url = index_url(df, base_url = r"https://www.sec.gov/Archives/")
        #Downloading index
        new_base_url, reports = download_summary(df_url)
        #Showing all reports for company
        all_reports = show_statements(reports)
        #Selecting 4 reports
        selected_reports = select_reports(all_reports)
        #Selecting URL's for each report
        urls = statements_urls(new_base_url, reports, selected_reports)
        #Downloading statements
        stm_data = statements_data(urls)
        #Converting statements into Pandas Dataframes
        statements_dict = financial_statements(stm_data)
        #Loading dictionary containing all statements for all input companies:
        key = df.Company_Name.unique()[0]
        company_statements_dict[key] = statements_dict
        
        
    return df, company_statements_dict

#### Run the following Function and pick the statements you want to download

In [26]:
df, out = run_all()

Welcome to ValuTrack Query Managemnt Tool
QMT lets you query the Edgar Index by Company Name, SIC Code, and Date
----------------------------------------------------------------------------------------
In the next steps you will be asked to formulate a query to retrieve available financial statements
You can add as many search conditions as you would like
The query will search for index entries matching all conditions entered
If you only wish to search by SIC code, leave the steps for Company Name and Year blank
----------------------------------------------------------------------------------------
Please make sure to either enter some Companies or SIC codes - Not Both!
----------------------------------------------------------------------------------------
Step 1: Company Names
Please enter company names in list format with names in quotes and separated by commas
For example: ["MINT LEASING INC", "NewStar Financial, Inc."]
Please enter number of companies you want to search for:


KeyboardInterrupt: 

In [31]:
out['MINT LEASING INC']['Consolidated Balance Sheets']

,"Dec. 31, 2013","Dec. 31, 2012"
Consolidated Balance Sheets (USD $),,
Investment in sales-type leases net of allowance of 445214 and 305174 respectively,18252772,22661504.0
Vehicle inventory,499963,479834.0
Property and equipment net,5431,27183.0
Other assets,4629,4629.0
TOTAL ASSETS,19085590,24067944.0
Accounts payable and accrued liabilities,1135000,846730.0
Credit facilities,10795049,21161332.0
Notes payable to related parties,1211200,1311564.0
Derivative Liability,1933501,NaN


In [29]:
df

,CIK,Company_Name,Filing_Type,Filing_Date,URL_txt,URL_html,SIC
0,1124127,MINT LEASING INC,10-K,2014-03-31,edgar/data/1124127/0001580695-14-000160.txt,edgar/data/1124127/0001580695-14-000160-index....,6172


In [ ]:
#Tests

In [ ]:
print('Financial statements for the followign companies have been downloaded:')
print(out.keys())

In [ ]:
print('The following statements for each company have been downloaded for AA:')
print(out['MINT LEASING INC'].keys())

In [ ]:
print('The following statements for each company have been downloaded for Amazon:')
print(out['MINT LEASING INC'].keys())

American Airlines

In [ ]:
out['MINT LEASING INC']['Consolidated Balance Sheets']

In [ ]:
out['AMERICAN AIRLINES INC']['Consolidated Balance Sheets']

In [ ]:
out['AMERICAN AIRLINES INC']['Consolidated Statements of Cash Flows']

In [ ]:
out['AMERICAN AIRLINES INC']["Consolidated Statements of Stockholders' Equity"]

Amazon

In [ ]:
out['AMAZON COM INC']['Consolidated Statements of Cash Flows']

In [ ]:
out['AMAZON COM INC']['Consolidated Statements of Comprehensive Income']

In [ ]:
out['AMAZON COM INC']['Consolidated Balance Sheets']

In [ ]:
out['AMAZON COM INC']["Consolidated Statements of Stockholders' Equity"]